<a href="https://colab.research.google.com/github/T-King-00/Gp-AutomationOfBaTasks/blob/tony/extractActorsNounsVerbs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip3 install PyGithub
from github import Github

import requests
response = requests.get('https://raw.githubusercontent.com/T-King-00/Gp-AutomationOfBaTasks/tony/university.txt')
file = response.text


! pip install -U spacy
from spacy.language import Language
import spacy
from spacy.matcher import Matcher
from spacy.symbols import nsubj, VERB



In [59]:
@Language.component("custom_sentencizer")
def custom_sentencizer(doc):
    for i, token in enumerate(doc[:-1]):
    
        if token.text == "."  :
          doc[i + 1].is_sent_start = True
        elif  token.text == "As":
          doc[i].is_sent_start = True
        else:
            # Explicitly set sentence start to False otherwise, to tell
            # the parser to leave those tokens alone
            doc[i + 1].is_sent_start = False
    return doc

nlp = spacy.load("en_core_web_sm")
nlp.add_pipe("custom_sentencizer", before="parser")  # Insert before the parser


/usr/local/lib/python3.8/dist-packages/spacy/util.py:877: UserWarning: [W095] Model 'en_core_web_sm' (3.4.1) was trained with spaCy v3.4 and may not be 100% compatible with the current version (3.5.0). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


<function __main__.custom_sentencizer(doc)>

In [61]:
doc = nlp(file)
sentences=[]
for sent in doc.sents:
  sentences.append(sent.text)

In [47]:
class Actor():

  def __init__(self,actorName):
   self.name=actorName
   self.usecases=[]
  def addUseCase(self,useCase):
    self.usecases.append(useCase)

###############################

In [48]:
actors=[]
for sentenceObj in sentences:
  objNlp=nlp(sentenceObj)
  ### this part get compound nouns (actors))
  for chunk in objNlp.noun_chunks:
    actorObj=Actor(chunk.text)
    if any(obj.name == actorObj.name for obj in actors):
      break
    else:
      actors.append(actorObj)
    break



print("Actors are #################")
print(len(actors))
for act in actors:
  print(act.name)


Actors are #################
9
a participant
an instructor
a site visitor
a search engine
a site owner
a trainer
a buyer
a user
a company


In [49]:
##### nouns 
nouns=[]
for sentenceObj in sentences:
  objNlp=nlp(sentenceObj)
  ### this part get compound nouns (actors))
  for i,chunk in enumerate(objNlp.noun_chunks):
    if i==0:
      continue;
    if chunk.text not in nouns:
      nouns.append(chunk.text)
    else:
      continue

for x in nouns:
  print(x)

I
the site
the midst
a quiz
video
my progress
a forum
a course
questions
topics
the instructor
other participants
announcements
a project “home page
a Scheduled Course
participants
thinks
chapter
an instructor
date
news
quizzes
their learning
part
feedback
material
the previous video(s
it
quiz responses
the database
people
various questions
that
which quizzes
the course outline
a comment
the world
what
each quiz question
the question
an answer
a different order
an FAQ
all my questions
answers
text
a site map
all pages
all feedback
historical or aggregate data
my courses
my account
my current information
a list
all courses
the PDUs
each
me
access
that historical information
someone
videos
those videos
a video course
those quizzes
they
the elements
course
them
an amazing course
a discussion forum
some
not all courses
the schedule
prospective participants
what course
elements
the publication date
course elements
a scheduled course
items
subtitles
any video
the thumbnail image
each video
t

In [64]:
##### verbs 
verbs=[]
for sentenceObj in sentences:
  objNlp=nlp(sentenceObj)
  ### this part get verbs)
  for i,tok in enumerate(objNlp):
    if tok.pos_=="VERB" and tok.lemma_ not in verbs:
      verb=tok.lemma_
      verbs.append(verb)
    else:
      continue

for x in verbs:
  print(x)

want
shut
’m
interrupt
participate
discuss
post
tell
delay
read
keep
create
assess
take
get
learn
see
pass
finish
store
do
fix
word
view
know
need
re
-
leave
think
receive
navigate
submit
press
answer
watch
index
log
manage
reflect
earn
have
call
upload
include
ensure
sequence
assemble
indicate
add
present
set
become
like
select
watermarke
make
offer
buy
give
customise
put
purchase
pay
prove
announce
subscribe
sign
control
understand
mark
complete
decide
begin
touch
show
display
respond
browse
pick
maintain
use
list
relate
look
opt
’
link
share
export
perform


In [68]:
##### compund verbs  phrasalverbs
matcher=Matcher(nlp.vocab)
compoundVerbs=[]

for sentenceVar in sentences: 
  v = sentenceVar.find("so that")
  sentenceVar=sentenceVar[0:v]
  x=nlp(sentenceVar)

  pattern0 = [[{"LOWER": "want"}, {"LOWER": "to" },{"POS":"VERB"},{"POS":"DET","OP":"*"},{"POS":"NOUN"}]
              ,[{"LOWER": "make"}, {"POS":"NOUN"},{"POS":"NOUN"}]]

  pattern1=[{"POS":"VERB"},{"POS":"NOUN"}]
  pattern2=[[{"POS":"VERB"},{"POS":"DET"},{"POS":"NOUN"}],[{"POS":"VERB"},{"LOWER":"the"},{"POS":"NOUN"}]]

  pattern3=[{"POS":"NOUN"},{"POS":"NOUN"}]
  matcher.add("verbPhrase", [pattern1])
  matcher.add("verbPhrase2", pattern2)


  matches = matcher(x)
  for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]  # Get string representation
      span = x[start:end]  # The matched span
      if span.text.find("want the")>-1:
        continue
      compoundVerbs.append(span)
      
compoundVerbs

[post announcements,
 read announcements,
 create quizzes,
 take a quiz,
 pass a quiz,
 want quiz,
 see which quizzes,
 viewing the course,
 leave a comment,
 receive feedback,
 navigate a quiz,
 submitting an answer,
 read an FAQ,
 get answers,
 view a site,
 view all feedback,
 view a list,
 upload videos,
 create a quiz,
 sequence the elements,
 indicate the schedule,
 set the publication,
 scheduled course,
 add subtitles,
 set the thumbnail,
 want all videos,
 set the prices,
 set quantity,
 create discount,
 put courses,
 receive a receipt,
 view the license,
 manage a set,
 purchase a site,
 watch any video,
 want each course,
 view some videos,
 set a mode,
 shown a page,
 learning credits,
 complete the course,
 earn a certificate,
 finishing a course,
 earn a badge,
 completed a course,
 leave a comment,
 create a description,
 browse a catalog,
 maintain a bio,
 see a list,
 related courses,
 viewing a course,
 shown an evaluation,
 completing a course,
 receiving an email,


In [ ]:

import re
class UserStory():
  def __init__(self):
   self.actors=[]
   self.usecases=[]
 
  def addActor(self,text):
    if text not in self.actors :
      self.actors.append(text)

  def checkSimilarityB2UseCases(self,t1):
    for x in self.usecases:
      print(t1,"---",x)
      v = re.search(t1,x)
      if v :
        return False

    return True

  def addUseCase(self,text):
    if text not in self.usecases and text!="want":
      v = text.find("want the")
      if v >-1:
        return 0
      if self.checkSimilarityB2UseCases(text):
        self.usecases.append(text)
 


   